### Install required packages

In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer  as Vader
from decimal import Decimal
import sys

import json
import boto3

### Access twitter APIs using consumer key, consumer secret, access token, access secret.

In [2]:
#consumer key, consumer secret, access token, access token secret.

ckey = " "
csecret = " "
atoken = " "
asecret= " "

### Stop after import $n$ tweets 

In [3]:
DynaDB = boto3.resource('dynamodb')
maproverTable = DynaDB.Table('tweets-by-location')

In [5]:
class listener(StreamListener):
    def __init__(self, api=None):
        super(StreamListener, self).__init__()
        self.num_tweets = 0

    def on_data(self, data):
        all_data = json.loads(data)
        tweet = all_data["text"]
        
        #------------------------------------
        #   Sentiment Analysis - output:tweet, sent_score, sent_status
        #------------------------------------
        sid = Vader()
        sent_score = sid.polarity_scores(tweet)
        
        if sent_score['pos']>0 and sent_score['compound']>0:
            sent_status = 'positive'
        elif sent_score['neg']>0 and sent_score['compound']<0:
            sent_status = 'negative'
        else:
            sent_status = 'neutral'
        
        
        #----------------------------------------------------------------
        #  Extract lat. long. (mass center)
        #----------------------------------------------------------------
        sent_score = sent_score['compound']
        coor = all_data['place']['bounding_box']['coordinates'][0]
        longitude = (coor[0][0] + coor[3][0])/2
        latitude = (coor[0][1] + coor[1][1])/2
     
        #------------------------------------------------------
        #   Save tweet information to DynamoDB
        #------------------------------------------------------
        
        TweetInfo = {
                        'UserID': all_data["id_str"],
                        'Created_Time':all_data["created_at"],
                        'Text': tweet,
                        'Longitude': longitude,
                        'Latitude': latitude,
                        'Sent_Score': sent_score,
                        'Sent_Status': sent_status                                    
                    }                     
        
        TweetInfo = json.dumps(TweetInfo)
        TweetInfo = json.loads(TweetInfo, parse_float=Decimal)
        
        maproverTable.put_item(Item = TweetInfo)
        
        self.num_tweets += 1
        if self.num_tweets < num_tweets:
            return True
        else:
            return False


    def on_error(self, status):
        print(status)

        
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())

num_tweets = 20
long_lat = [-77.00, 38.87, -75.00, 39.81]   # Washington D.C. USA [long_1, lat_1, long_2, lat_2]
twitterStream.filter(locations = long_lat)